# Reverse numerical methods applied to channel flow

Our goals here, in general, are to find new valid equations that describe fluid flow. 
We will try random PDEs and use the steady RANS equations to attempt to solve for the Reynolds
stresses in terms of the other quantities we know.


## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Now, let's add some terms and assume that their coefficients are isotropic:

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= 
  A (\nabla P)^2
+ B \nabla P^2
+ C (\vec{U} \cdot \nabla) \vec{U}^2
+ D \nabla^2 \vec{U}^2
+ E \nabla K
$$

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial x} + \nu \left( \frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2} + \frac{\partial^2 U}{\partial z^2} \right)
+ R_x
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} + W \frac{\partial V}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial y} + \nu \left( \frac{\partial^2 V}{\partial x^2}
+ \frac{\partial^2 V}{\partial y^2} + \frac{\partial^2 V}{\partial z^2} \right)
+ R_y
$$

$$
U \frac{\partial W}{\partial x} 
+ V \frac{\partial W}{\partial y} + W \frac{\partial W}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial z} + \nu \left( \frac{\partial^2 W}{\partial x^2}
+ \frac{\partial^2 W}{\partial y^2} + \frac{\partial^2 W}{\partial z^2} \right)
+ R_z
$$


## Quantities available from JHTDB

* Velocity (`GetVelocity`)
* Pressure (`GetPressure`)
* Velocity gradient (`GetVelocityGradient`)
* Pressure gradient (`GetPressureGradient`)
* Velocity Laplacian (`GetVelocityLaplacian`)
* Velocity Hessian (`GetVelocityHessian`)
* Pressure Hessian (`GetPressureHessian`)


## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

In [ ]:
import numpy as np
import pyJHTDB
from pyJHTDB.dbinfo import channel as info

npoints = 128
nskip = 4

# x = np.random.random(size=(npoints, info['ny']/nskip-1, 3)).astype(np.float32)

# x = np.ones((1, 3))*0.5
x = [(0.5, 0.0, 0.5)]

# x[..., 0] *= info['lx']
# x[..., 1] = 0
# # x[..., 1] *= info['dy'][::nskip][None, :x.shape[1]]
# # x[..., 1] += info['ynodes'][::nskip][None, :x.shape[1]]
# x[..., 2] *= info['lz']

print(info["xnodes"].min(), info["xnodes"].max(),
      info["ynodes"].min(), info["ynodes"].max(),
      info["znodes"].min(), info["znodes"].max()
     )

x = np.asarray(x, dtype=np.float32)
time = np.linspace(0, 10.0, 30)

In [ ]:
from pyJHTDB import libJHTDB

# interp_info = [(44, 'FD4Lag4'),
#                (104, 'M1Q4'),
#                (208, 'M2Q8'),
#                (214, 'M2Q14')]
# divu = []

lJHTDB = libJHTDB()
lJHTDB.initialize()

"""
for ii in interp_info:
    gradu = lJHTDB.getData(
                0.0,
                x,
                sinterp = ii[0],
                tinterp = 0,
                data_set = info['name'],
                getFunction = 'getVelocityGradient')
    divu.append(np.average(np.abs(
                        gradu[..., 0] +
                        gradu[..., 4] +
                        gradu[..., 8]), axis = 0) / 
                np.average(np.sqrt(
                        gradu[..., 0]**2 +
                        gradu[..., 4]**2 +
                        gradu[..., 8]**2), axis = 0))
"""

u = []

for t in time:
    u.append(lJHTDB.getData(t, x, getFunction="getVelocity", tinterp="PCHIPInt",
                            sinterp=104, data_set=info["name"]))

lJHTDB.finalize()

In [ ]:
u = np.asarray(u)
u = u[:, 0]
print(u.shape)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (10, 6))
ax = fig.add_subplot(111)

# for i in range(len(divu)):
#     ax.plot(info['ynodes'][::nskip][:divu[i].shape[0]], divu[i], label=interp_info[i][1])

# ax.set_ylabel("Divergence")
# ax.legend(loc='best')
# ax.set_ylim(0, 0.255)

for n, label in enumerate(["$u$", "$v$", "$w$"]):
    plt.plot(time, u[:, n], label=label)
plt.xlabel("Time (s)")
plt.ylabel("Velocity")
plt.legend(loc="best")
plt.show()